In [5]:
import pandas as pd
import numpy as np
from nsepython import equity_history
from datetime import datetime, timedelta

In [32]:
class DataLoader:
    req_columns = ['CH_TIMESTAMP', 'CH_SYMBOL', 'CH_TRADE_HIGH_PRICE', 'CH_TRADE_LOW_PRICE', 'CH_OPENING_PRICE', 'CH_CLOSING_PRICE', 'CH_LAST_TRADED_PRICE', 'CH_PREVIOUS_CLS_PRICE', 'CH_TOT_TRADED_QTY', 'CH_52WEEK_HIGH_PRICE', 'CH_52WEEK_LOW_PRICE']
    new_column_names = ['date', 'symbol', 'high', 'low', 'open', 'close', 'ltp', 'prev_close', 'volume', 'high_52w', 'low_52w']

    @staticmethod
    def load_data(script_name, start_date, end_date, series="EQ"):
        df = equity_history(script_name, series, start_date, end_date)[DataLoader.req_columns]
        df.columns = DataLoader.new_column_names
        return df

In [225]:
def load_volume_sma(df):
    df['vol_sma_10d'] = df.volume.rolling(window=10).mean()
    return df

def load_emas(df):
    ema_spans = [9, 21, 30, 50, 100, 12, 26]
    for i in ema_spans:
        df[f'ema_{i}d'] = df.close.ewm(span=i).mean()
    return df

def load_macd(df):
    df['macd'] = df.ema_12d - df.ema_26d
    df['macd_9d_signal'] = df.macd.ewm(9).mean()
    return df
    
def load_bollinger_bands(df):
    df['sma_20d'] = df.close.rolling(window=20).mean()
    df['bb_upper'] = df.sma_20d + 2 * df.close.rolling(20).std()
    df['bb_lower'] = df.sma_20d - 2 * df.close.rolling(20).std()
    return df

In [205]:
script_name = 'INFY'
series = 'EQ'
end_date = datetime.now().date()
start_date = end_date - timedelta(days = 365)

In [227]:
df = DataLoader.load_data(
    script_name, 
    start_date.strftime("%d-%m-%Y"), 
    end_date.strftime("%d-%m-%Y"), 
    series).sort_values('date').drop_duplicates()

In [228]:
for fn in [load_volume_sma, load_emas, load_macd, load_bollinger_bands]:
    df = fn(df)

In [229]:
df.tail(30)

,date,symbol,high,low,open,close,ltp,prev_close,volume,high_52w,low_52w,vol_sma_10d,ema_9d,ema_21d,ema_30d,ema_50d,ema_100d,ema_12d,ema_26d,macd,macd_9d_signal,sma_20d,bb_upper,bb_lower
57,2022-07-21,INFY,1535.45,1503.10,1517.00,1532.90,1532.00,1515.30,5008061,1953.9,1367.15,4944856.6,1487.262311,1476.899224,1478.939604,1499.664254,1561.314771,1482.620950,1477.219941,5.401008,-7.503321,1475.3400,1532.895635,1417.784365
3,2022-07-22,INFY,1540.00,1492.15,1534.00,1506.30,1506.00,1532.90,7020642,1953.9,1367.15,5281662.6,1491.069848,1479.572022,1480.704791,1499.924518,1560.211831,1486.263880,1479.374020,6.889861,-6.064003,1478.6000,1535.370424,1421.829576
4,2022-07-25,INFY,1517.15,1480.10,1480.10,1503.60,1502.00,1506.30,7859626,1953.9,1367.15,5597343.7,1493.575879,1481.756384,1482.181902,1500.068676,1559.077152,1488.930976,1481.168537,7.762439,-4.681359,1480.0500,1537.862100,1422.237900
5,2022-07-26,INFY,1496.65,1445.00,1496.65,1451.15,1450.85,1503.60,7268273,1953.9,1367.15,5853135.4,1485.090703,1478.973985,1480.179843,1498.150030,1556.914473,1483.118518,1478.944942,4.173576,-3.795865,1478.6000,1537.838652,1419.361348
6,2022-07-27,INFY,1473.00,1445.00,1451.00,1471.05,1468.20,1451.15,4517285,1953.9,1367.15,5778461.4,1482.282562,1478.253623,1479.590821,1497.087141,1555.194300,1481.261823,1478.360131,2.901692,-3.126110,1478.9900,1537.904966,1420.075034
7,2022-07-28,INFY,1524.45,1489.85,1496.40,1517.40,1517.20,1471.05,6636841,1953.9,1367.15,5826384.9,1489.306050,1481.812385,1482.030123,1497.883826,1554.437320,1486.821542,1481.251973,5.569569,-2.256542,1481.7650,1542.491081,1421.038919
8,2022-07-29,INFY,1555.70,1528.00,1532.80,1549.70,1546.00,1517.40,7009465,1953.9,1367.15,5965329.6,1501.384840,1487.983986,1486.395923,1499.916084,1554.342458,1496.495151,1486.322198,10.172954,-1.013592,1485.2800,1553.148041,1417.411959
9,2022-08-01,INFY,1564.00,1537.90,1564.00,1551.05,1551.50,1549.70,2983249,1953.9,1367.15,5680137.4,1511.317872,1493.717260,1490.567155,1501.921573,1554.276544,1504.888205,1491.116850,13.771355,0.464903,1488.4325,1562.414527,1414.450473
10,2022-08-02,INFY,1547.00,1528.35,1546.55,1543.60,1539.90,1551.05,4374635,1953.9,1367.15,5775746.9,1517.774298,1498.252055,1493.988630,1503.556207,1554.062846,1510.843866,1495.004491,15.839375,2.002350,1491.8150,1569.487972,1414.142028
11,2022-08-03,INFY,1570.00,1539.90,1549.00,1566.10,1569.90,1543.60,6011955,1953.9,1367.15,5869003.2,1527.439438,1504.420050,1498.640978,1506.009173,1554.303726,1519.344809,1500.270825,19.073985,3.709513,1495.5375,1580.015182,1411.059818


In [199]:
df['gain_pts'] = np.where(df.close > df.prev_close, (df.close - df.prev_close).abs(), 0)
df['loss_pts'] = np.where(df.close < df.prev_close, (df.close - df.prev_close).abs(), 0)

# df['rs_factor'] = (
#     pd.Series(
#         np.where(df.close > df.prev_close, df.close - df.prev_close, 0)
#     ).rolling(window=15).mean() /
#     pd.Series(
#         np.where(df.close < df.prev_close, df.close - df.prev_close, 0)
#     ).rolling(window=15).mean()
# )

df['gain_avg_14d'] = df.gain_pts.rolling(14).mean()
df['loss_avg_14d'] = df.loss_pts.rolling(14).mean()
# df['rs_factor'] = df.gain_pts.ewm(span=15, adjust=True).mean() / df.loss_pts.ewm(span=15, adjust=True).mean()

df['gain_avg_14d'] = (df.gain_avg_14d.shift(-1) * 13 + df.gain_pts) / 14
df['loss_avg_14d'] = (df.loss_avg_14d.shift(-1) * 13 + df.loss_pts) / 14

df['rs_factor'] = df.gain_avg_14d / df.loss_avg_14d

df['rsi'] = 100 - (100 / (1 + df.rs_factor))

df.tail(30)